# **Datamining Conversation Text**
Within this notebook, I'm going to explore my options for organizing and datamining the conversation text.

# Setup
The cells below will set up the rest of the notebook.

I'll start by configuring the kernel: 

In [ ]:
# Change the working directory 
%cd ..

# Enable the autoreload extension, which will automatically load in new code as it's written
%load_ext autoreload
%autoreload 2

Now I'll import some necessary modules:

In [ ]:
# General import statements
import json

# Third-party import statements
import pandas as pd

# ______
This is where the rest of the notebook starts. Go wild. 